In [1]:
import pandas as pd
import numpy as np

In [2]:
info = pd.read_csv('data/sports_retail/info.csv')
finance = pd.read_csv('data/sports_retail/finance.csv')
reviews = pd.read_csv('data/sports_retail/reviews.csv')
traffic = pd.read_csv('data/sports_retail/traffic.csv')
brands = pd.read_csv('data/sports_retail/brands.csv')

In [6]:
class display(object):
    """Display HTML representation of multiple objects"""
    template = """<div style="float: left; padding: 10px;">
    <p style='font-family:"Courier New", Courier, monospace'>{0}</p>{1}
    </div>"""
    def __init__(self, *args):
        self.args = args
        
    def _repr_html_(self):
        return '\n'.join(self.template.format(a, eval(a)._repr_html_())
                         for a in self.args)
    
    def __repr__(self):
        return '\n\n'.join(a + '\n' + repr(eval(a))
                           for a in self.args)

In [9]:
display('info.head(3)','finance.head(3)','reviews.head(3)','traffic.head(3)','brands.head(3)')

info.head(3)
                           product_name product_id  \
0                                   NaN     AH2430   
1  Women's adidas Originals Sleek Shoes     G27341   
2     Women's adidas Swim Puka Slippers     CM0081   

                                         description  
0                                                NaN  
1  A modern take on adidas sport heritage, tailor...  
2  These adidas Puka slippers for women's come wi...  

finance.head(3)
  product_id  listing_price  sale_price  discount  revenue
0     AH2430            NaN         NaN       NaN      NaN
1     G27341          75.99       37.99       0.5  1641.17
2     CM0081           9.99        5.99       0.4   398.93

reviews.head(3)
  product_id  rating  reviews
0     AH2430     NaN      NaN
1     G27341     3.3     24.0
2     CM0081     2.6     37.0

traffic.head(3)
  product_id         last_visited
0     AH2430  2018-05-19 15:13:00
1     G27341  2018-11-29 16:16:00
2     CM0081  2018-02-01 10:27:00

brands.head(3)
  product_id   brand
0     AH2430     NaN
1     G27341  Adidas
2     CM0081  Adidas

In [17]:
merged_df = pd.merge(info,finance,on='product_id',how='outer')
merged_df = pd.merge(merged_df,reviews,on='product_id',how='outer')
merged_df = pd.merge(merged_df,traffic,on='product_id',how='outer')
merged_df = pd.merge(merged_df,brands,on='product_id',how='outer')

In [19]:
merged_df.dropna(inplace=True)
merged_df.head()

,product_name,product_id,description,listing_price,sale_price,discount,revenue,rating,reviews,last_visited,brand
1,Women's adidas Originals Sleek Shoes,G27341,"A modern take on adidas sport heritage, tailor...",75.99,37.99,0.5,1641.17,3.3,24.0,2018-11-29 16:16:00,Adidas
2,Women's adidas Swim Puka Slippers,CM0081,These adidas Puka slippers for women's come wi...,9.99,5.99,0.4,398.93,2.6,37.0,2018-02-01 10:27:00,Adidas
3,Women's adidas Sport Inspired Questar Ride Shoes,B44832,"Inspired by modern tech runners, these women's...",69.99,34.99,0.5,2204.37,4.1,35.0,2018-09-07 20:06:00,Adidas
4,Women's adidas Originals Taekwondo Shoes,D98205,This design is inspired by vintage Taekwondo s...,79.99,39.99,0.5,5182.70,3.5,72.0,2019-07-18 15:26:00,Adidas
5,Women's adidas Sport Inspired Duramo Lite 2.0 ...,B75586,Refine your interval training in these women's...,47.99,19.20,0.6,1555.20,1.0,45.0,2019-01-30 12:09:00,Adidas


***
### What is the volume of products and average revenue for Adidas and Nike products based on price quartiles?

In [20]:
# Add price labels based on listing_price quartiles
merged_df['price_label'] = pd.qcut(merged_df['listing_price'],4,['Budget','Average','Expensive','Elite'])
merged_df.head()

,product_name,product_id,description,listing_price,sale_price,discount,revenue,rating,reviews,last_visited,brand,price_label
1,Women's adidas Originals Sleek Shoes,G27341,"A modern take on adidas sport heritage, tailor...",75.99,37.99,0.5,1641.17,3.3,24.0,2018-11-29 16:16:00,Adidas,Expensive
2,Women's adidas Swim Puka Slippers,CM0081,These adidas Puka slippers for women's come wi...,9.99,5.99,0.4,398.93,2.6,37.0,2018-02-01 10:27:00,Adidas,Budget
3,Women's adidas Sport Inspired Questar Ride Shoes,B44832,"Inspired by modern tech runners, these women's...",69.99,34.99,0.5,2204.37,4.1,35.0,2018-09-07 20:06:00,Adidas,Expensive
4,Women's adidas Originals Taekwondo Shoes,D98205,This design is inspired by vintage Taekwondo s...,79.99,39.99,0.5,5182.70,3.5,72.0,2019-07-18 15:26:00,Adidas,Expensive
5,Women's adidas Sport Inspired Duramo Lite 2.0 ...,B75586,Refine your interval training in these women's...,47.99,19.20,0.6,1555.20,1.0,45.0,2019-01-30 12:09:00,Adidas,Average


In [73]:
# Group by brand and price_label to get volume and mean revenue
adidas_vs_nike = merged_df.groupby(['brand','price_label']).agg({'price_label':'count','revenue':'mean'})
adidas_vs_nike

price_label      revenue
brand  price_label                          
Adidas Budget               538  2050.966580
       Average              599  2982.297429
       Expensive            707  4599.578600
       Elite                533  8424.178574
Nike   Budget               321  1664.329595
       Average                8   675.592500
       Expensive             43   472.739070
       Elite                124  1418.420484

***
### Do any differences exist between the word count of a product's description and its mean rating?

In [29]:
print(merged_df['description'].str.len().min())
print(merged_df['description'].str.len().max())

14
687


In [30]:
word_limits = [0,99,199,299,399,499,599,699]
limit_bins = ['99','199','299','399','499','599','699']

merged_df['word_limit'] = merged_df['description'].str.len()

merged_df['word_limit'] = pd.cut(merged_df['word_limit'],bins=word_limits,labels=limit_bins)
merged_df.head()

,product_name,product_id,description,listing_price,sale_price,discount,revenue,rating,reviews,last_visited,brand,price_label,word_limit
1,Women's adidas Originals Sleek Shoes,G27341,"A modern take on adidas sport heritage, tailor...",75.99,37.99,0.5,1641.17,3.3,24.0,2018-11-29 16:16:00,Adidas,Expensive,199
2,Women's adidas Swim Puka Slippers,CM0081,These adidas Puka slippers for women's come wi...,9.99,5.99,0.4,398.93,2.6,37.0,2018-02-01 10:27:00,Adidas,Budget,199
3,Women's adidas Sport Inspired Questar Ride Shoes,B44832,"Inspired by modern tech runners, these women's...",69.99,34.99,0.5,2204.37,4.1,35.0,2018-09-07 20:06:00,Adidas,Expensive,299
4,Women's adidas Originals Taekwondo Shoes,D98205,This design is inspired by vintage Taekwondo s...,79.99,39.99,0.5,5182.70,3.5,72.0,2019-07-18 15:26:00,Adidas,Expensive,299
5,Women's adidas Sport Inspired Duramo Lite 2.0 ...,B75586,Refine your interval training in these women's...,47.99,19.20,0.6,1555.20,1.0,45.0,2019-01-30 12:09:00,Adidas,Average,299


In [31]:
merged_df.groupby('word_limit').agg({'rating':'mean','reviews':'count'})

,rating,reviews
word_limit,,
99,1.866667,6
199,3.188937,461
299,3.287108,1660
399,3.313765,603
499,3.396460,113
599,3.120000,15
699,3.653333,15


In [33]:
description = merged_df.groupby('word_limit',as_index=False).agg({'rating':'mean','reviews':'count'})
description

,word_limit,rating,reviews
0,99,1.866667,6
1,199,3.188937,461
2,299,3.287108,1660
3,399,3.313765,603
4,499,3.396460,113
5,599,3.120000,15
6,699,3.653333,15


***
### How does the volume of products and median revenue vary between clothing and footwear?

In [50]:
footwear_keywords = 'shoe*|foot*|trainer*'

In [56]:
# filter for footwear products
footwear = merged_df[merged_df['description'].str.contains(footwear_keywords)]
footwear.head()

,product_name,product_id,description,listing_price,sale_price,discount,revenue,rating,reviews,last_visited,brand,price_label,word_limit
1,Women's adidas Originals Sleek Shoes,G27341,"A modern take on adidas sport heritage, tailor...",75.99,37.99,0.5,1641.17,3.3,24.0,2018-11-29 16:16:00,Adidas,Expensive,199
2,Women's adidas Swim Puka Slippers,CM0081,These adidas Puka slippers for women's come wi...,9.99,5.99,0.4,398.93,2.6,37.0,2018-02-01 10:27:00,Adidas,Budget,199
3,Women's adidas Sport Inspired Questar Ride Shoes,B44832,"Inspired by modern tech runners, these women's...",69.99,34.99,0.5,2204.37,4.1,35.0,2018-09-07 20:06:00,Adidas,Expensive,299
4,Women's adidas Originals Taekwondo Shoes,D98205,This design is inspired by vintage Taekwondo s...,79.99,39.99,0.5,5182.70,3.5,72.0,2019-07-18 15:26:00,Adidas,Expensive,299
5,Women's adidas Sport Inspired Duramo Lite 2.0 ...,B75586,Refine your interval training in these women's...,47.99,19.20,0.6,1555.20,1.0,45.0,2019-01-30 12:09:00,Adidas,Average,299


In [68]:
product_ids_shoe = footwear.product_id
product_ids_clothing = ~merged_df['product_id'].isin(product_ids_shoe)
clothing = merged_df[product_ids_clothing]
clothing.head() 

,product_name,product_id,description,listing_price,sale_price,discount,revenue,rating,reviews,last_visited,brand,price_label,word_limit
71,Men's adidas Toe Side II Slippers,CM0068,The adidas casual sandals for men with a strik...,12.99,7.79,0.4,827.30,1.5,59.0,2019-01-18 01:16:00,Adidas,Budget,299
100,Men's adidas Running Nepton 2.0 Shoes,CL7470,Energize your run while staying comfortable wi...,55.99,27.99,0.5,251.91,4.0,5.0,2018-10-23 12:53:00,Adidas,Average,299
105,Men's adidas Toe Side II Slippers,CM0069,The adidas casual sandals for men with a strik...,12.99,7.79,0.4,308.48,3.0,22.0,2018-05-24 12:24:00,Adidas,Budget,299
114,Men's adidas Running Nepton 2.0 Shoes,CL7471,Energize your run while staying comfortable wi...,55.99,27.99,0.5,1813.75,3.8,36.0,2020-01-14 16:44:00,Adidas,Average,299
136,Women's Originals Sleek Super Shoes,EF4954,These patent leather adidas Sleek Super Shoes ...,79.99,55.99,0.3,9675.07,2.2,96.0,2018-07-29 08:24:00,Adidas,Expensive,199


In [71]:
# Create product_type DataFrame

product_type = pd.DataFrame({
                                'clothing volume':[len(clothing)],
                                'median revenue of clothing':[clothing.revenue.median()],
                                'footwear volume':[len(footwear)],
                                'median revenue of footwear':[footwear.revenue.median()]

})

In [72]:
product_type

,clothing volume,median revenue of clothing,footwear volume,median revenue of footwear
0,439,683.73,2434,3073.3
